# Multi-Agent Negotiation System Tutorial 🤝

Welcome to an exciting exploration of AI-powered negotiations! In this tutorial, we'll learn how to create AI agents that can negotiate with each other in realistic scenarios.

## What You'll Learn 🎯

In this comprehensive tutorial, you'll discover:

1. **🤖 Multi-Agent Systems**: How different AI agents with conflicting goals interact
2. **💰 Negotiation Dynamics**: How AI can simulate realistic bargaining scenarios
3. **🎭 Agent Personalities**: How to give AI agents different negotiation styles
4. **📊 Game Theory**: Understanding strategic decision-making in AI
5. **🔍 Analysis & Moderation**: How AI can analyze and judge negotiations
6. **🏗️ System Architecture**: Building a complete multi-agent application

Let's dive into the fascinating world of AI negotiations! 🚀


## Setup and Prerequisites 🛠️

Before we start creating our negotiation agents, let's make sure we have everything we need:

### What We Need:
- **Ollama** running on your computer
- **Python packages**: ollama, json, re, datetime
- **AI Models**: At least one model like phi3 or phi4

Let's install the required packages and set up our imports:


In [ ]:
# Install required packages
%pip install ollama

# Import all necessary libraries
import json
import os
import re
import time
from datetime import datetime
from typing import Dict, Any, Tuple, Optional
from ollama import chat, list as ollama_list

print("✅ All packages installed and imported!")
print("🎯 Ready to build our negotiation system!")


## Part 1: Configuration and Scenarios 📋

Every negotiation needs a setting! Let's start by defining our configuration and the scenarios where our AI agents will negotiate.


In [ ]:
# Configuration settings for our negotiation system
DEFAULT_MODEL = "phi3"  # Default AI model to use
DEFAULT_TEMP = 0.8      # Higher temperature for more creative negotiations
DEFAULT_SEED = 42       # For reproducible results

# Negotiation scenarios - each one is a different setting with different dynamics
NEGOTIATION_SCENARIOS = {
    "yard_sale_lamp": {
        "name": "Local Yard Sale",
        "item_name": "a dusty but charming vintage lamp",
        "list_price": 40,
        "seller_personality": "friendly but a little attached to their old items",
        "seller_min_price": 20,          # Secret: Seller won't go below this
        "buyer_desire_level": "medium",
        "buyer_target_price": 25,        # Secret: Buyer's ideal price
        "buyer_max_price": 35,           # Secret: Buyer's absolute limit
    },
    "fish_market": {
        "name": "Busy Fish Market",
        "item_name": "the last fresh tuna of the day",
        "list_price": 75,
        "seller_personality": "a bit gruff and in a hurry to close up shop",
        "seller_min_price": 55,
        "buyer_desire_level": "high",
        "buyer_target_price": 60,
        "buyer_max_price": 70,
    },
    "pokemon_card": {
        "name": "Comic Book Store",
        "item_name": "a rare, holographic Charizard card",
        "list_price": 500,
        "seller_personality": "a savvy and expert collector who knows the card's value",
        "seller_min_price": 420,
        "buyer_desire_level": "very high",
        "buyer_target_price": 440,
        "buyer_max_price": 480,
    }
}

def list_scenarios():
    """Display all available negotiation scenarios in a nice format"""
    print("🎭 Available Negotiation Scenarios:")
    print("=" * 60)
    for key, scenario in NEGOTIATION_SCENARIOS.items():
        print(f"📍 {key}: A negotiation for '{scenario['item_name']}' at a {scenario['name']}")
        print(f"   💰 Listed Price: ${scenario['list_price']}")
        print(f"   🎭 Seller: {scenario['seller_personality']}")
        print(f"   ❤️ Buyer Desire: {scenario['buyer_desire_level']}")
        print()
    print("=" * 60)

def get_scenario(scenario_key: str) -> dict:
    """Get a specific scenario by its key"""
    return NEGOTIATION_SCENARIOS.get(scenario_key)

# Let's see our scenarios!
list_scenarios()

# Example: Look at the yard sale scenario in detail
yard_sale = get_scenario("yard_sale_lamp")
print("🔍 Detailed look at yard_sale_lamp scenario:")
print(json.dumps(yard_sale, indent=2))


## Part 2: Ollama Communication 🤖

Before we create our agents, let's build the foundation - a way to communicate with Ollama models. This is the core function that all our agents will use to "think" and respond.


In [ ]:
def ollama_query(ollama_model: str, 
                 prompt_to_LLM: str, 
                 temperature: float, 
                 seed: int = 0,
                 num_ctx: int = 4096,
                 top_k: int = 40,
                 top_p: float = 0.9,
                 min_p: float = 0.05,
                 repeat_penalty: float = 1.1) -> str:
    """
    Send a prompt to an Ollama model and get a response.
    
    This is the core communication function that all our agents will use
    to interact with the AI models.
    
    Args:
        ollama_model: Name of the model to use (e.g., 'phi3', 'llama3')
        prompt_to_LLM: The instruction/question to send to the model
        temperature: How creative the response should be (0.0-1.0)
        seed: For reproducible results
        num_ctx: Context window size
        top_k, top_p, min_p: Sampling parameters for response generation
        repeat_penalty: Penalty for repeating words
    
    Returns:
        The model's response as a string
    """
    try:
        response = chat(
            model=ollama_model,
            options={
                "temperature": temperature,
                "seed": seed,
                "num_ctx": num_ctx,
                "top_k": top_k,
                "top_p": top_p,
                "min_p": min_p,
                "repeat_penalty": repeat_penalty
            },
            messages=[{
                'role': 'user',
                'content': prompt_to_LLM
            }]
        )
        return response['message']['content']
    except Exception as e:
        return f"Error communicating with model {ollama_model}: {e}"

def print_ollama_models():
    """Show what models are available in Ollama"""
    try:
        response = ollama_list()
        print("🤖 Available Ollama Models:")
        print("=" * 40)
        for model in response['models']:
            name = model['model']
            size_mb = model['size'] / (1024 * 1024)
            print(f"📦 {name} ({size_mb:.1f} MB)")
        print("=" * 40)
    except Exception as e:
        print(f"❌ Error listing models: {e}")
        print("Make sure Ollama is running!")

# Let's see what models we have available
print_ollama_models()

# Test our communication function
print("\\n🧪 Testing communication with AI model:")
try:
    test_response = ollama_query(
        ollama_model="phi3",
        prompt_to_LLM="Hello! Please introduce yourself briefly as a negotiation expert.",
        temperature=0.7,
        seed=42
    )
    print(f"🤖 AI Response: {test_response}")
except Exception as e:
    print(f"❌ Error: {e}")
    print("Make sure you have the 'phi3' model installed: ollama pull phi3")


## Part 3: Crafting Negotiation Prompts 📝

The secret to realistic AI negotiations is in the prompts! Let's create detailed instructions that will make our AI agents behave like real negotiators with different goals and personalities.


In [ ]:
# Prompt templates for our negotiation agents
# These are the "scripts" that tell each AI how to behave

# 💰 SELLER AGENT PROMPT
SELLER_PROMPT = """You are a Seller at a {scenario_name}.
You are selling: '{item_name}'.
The listed price is: ${list_price}.

Your secret goal is to get the highest price possible, but you absolutely CANNOT sell for less than your secret walk-away price of ${min_price}.
Your personality is: {personality}.

Below is the negotiation history so far. Continue the conversation naturally, and make your next offer.
Do not sound like a robot/AI or over pompous. Be concise and to the point, use no more than 100 words or less.
Your response MUST end with your new offer price on its own line, formatted exactly like this:
Price: $XX

Negotiation History:
{chat_history}

Your Response:"""

# 🛒 BUYER AGENT PROMPT  
BUYER_PROMPT = """You are a Buyer at a {scenario_name}.
You want to buy: '{item_name}'.
The seller's listed price is: ${list_price}.

Your secret goal is to get the best deal possible. Your target price is ${target_price}, but you absolutely CANNOT pay more than your secret maximum price of ${max_price}.
Your desire for this item is: {desire_level}.

Below is the negotiation history so far. Continue the conversation naturally, and make your next counter-offer.
Do not sound like a robot/AI or over pompous. Be concise and to the point, use no more than 100 words or less.
Your response MUST end with your new offer price on its own line, formatted exactly like this:
Price: $XX

Negotiation History:
{chat_history}

Your Response:"""

# ⚖️ MODERATOR AGENT PROMPT
MODERATOR_PROMPT = """You are a world-class negotiation expert analyzing a transaction.
The negotiation transcript is provided below.

Here is the public and secret information:
- Item: '{item_name}'
- Final Deal Price: ${final_price}
- Seller's Secret Minimum Price: ${seller_min_price}
- Buyer's Secret Target Price: ${buyer_target_price}
- Buyer's Secret Maximum Price: ${buyer_max_price}

Based on all this information, who got the better deal and why?
Do not sound like a robot/AI and be concise and to the point, use no more than 150 words or less.
Provide a brief, expert analysis of the negotiation, including what tactics were used effectively or ineffectively.

Transcript:
{transcript}

Your Analysis:"""

print("✅ All negotiation prompts defined!")
print("\\n🔍 Let's see how a seller prompt looks when filled in:")

# Example: Fill in the seller prompt for the yard sale scenario
example_scenario = get_scenario("yard_sale_lamp")
example_prompt = SELLER_PROMPT.format(
    scenario_name=example_scenario['name'],
    item_name=example_scenario['item_name'],
    list_price=example_scenario['list_price'],
    min_price=example_scenario['seller_min_price'],
    personality=example_scenario['seller_personality'],
    chat_history="(No previous conversation)"
)

print("\\n" + "="*60)
print("EXAMPLE SELLER PROMPT:")
print("="*60)
print(example_prompt)


## Part 4: Creating Negotiation Agents 🎭

Now let's create our AI agents! Each agent will have different goals, personalities, and strategies. We'll build a base Agent class and then create specialized Buyer, Seller, and Moderator agents.


In [ ]:
class Agent:
    """
    Base class for all negotiation agents.
    
    This provides common functionality that all agents need:
    - Communication with AI models
    - Error handling
    - Price parsing from responses
    """
    
    def __init__(self, model: str, temp: float, seed: int):
        self.model = model
        self.temp = temp
        self.seed = seed
        print(f"🤖 Created {self.__class__.__name__} using model: {model}")

    def _query_llm(self, prompt: str) -> str:
        """
        Send a prompt to the AI model with retry logic.
        
        This method handles communication errors gracefully by trying
        with a different seed if the first attempt fails.
        """
        try:
            return ollama_query(self.model, prompt, self.temp, self.seed)
        except Exception as e:
            print(f"⚠️ Error querying model {self.model}: {e}. Retrying...")
            try:
                # Try again with a different seed
                return ollama_query(self.model, prompt, self.temp, self.seed + 1)
            except Exception as e2:
                print(f"❌ Retry failed for {self.model}: {e2}")
                return "Agent encountered an error and cannot respond."

    def parse_price(self, response: str) -> Tuple[Optional[float], str]:
        """
        Extract the price from the agent's response.
        
        Agents must end their responses with "Price: $XX" so we can
        track the negotiation progress.
        
        Args:
            response: The full response from the AI agent
            
        Returns:
            Tuple of (price, cleaned_message) or (None, original_message)
        """
        # Look for "Price: $XX" pattern (case insensitive)
        price_match = re.search(r"Price: \\$?([0-9]+\\.?[0-9]*)", response, re.IGNORECASE)
        
        if price_match:
            price = float(price_match.group(1))
            # Remove the price line from the message for cleaner display
            cleaned_response = response[:price_match.start()].strip()
            return price, cleaned_response
        
        # If no price found, return None and the original response
        return None, response

class SellerAgent(Agent):
    """
    Represents the Seller in the negotiation.
    
    The seller's goal is to get the highest price possible while
    staying above their secret minimum price.
    """
    
    def __init__(self, model: str, temp: float, seed: int, scenario: dict):
        super().__init__(model, temp, seed)
        self.scenario = scenario
        print(f"💰 Seller initialized - Min price: ${scenario['seller_min_price']} (secret)")

    def act(self, chat_history: str) -> Tuple[Optional[float], str]:
        """
        Generate the seller's next move in the negotiation.
        
        Args:
            chat_history: Previous conversation between buyer and seller
            
        Returns:
            Tuple of (price_offer, message)
        """
        # Fill in the seller prompt with scenario details and history
        prompt = SELLER_PROMPT.format(
            scenario_name=self.scenario['name'],
            item_name=self.scenario['item_name'],
            list_price=self.scenario['list_price'],
            min_price=self.scenario['seller_min_price'],
            personality=self.scenario['seller_personality'],
            chat_history=chat_history
        )
        
        # Get response from AI and parse the price
        response = self._query_llm(prompt)
        return self.parse_price(response)

class BuyerAgent(Agent):
    """
    Represents the Buyer in the negotiation.
    
    The buyer's goal is to get the best deal possible while
    staying under their secret maximum price.
    """
    
    def __init__(self, model: str, temp: float, seed: int, scenario: dict):
        super().__init__(model, temp, seed)
        self.scenario = scenario
        print(f"🛒 Buyer initialized - Target: ${scenario['buyer_target_price']}, Max: ${scenario['buyer_max_price']} (secret)")

    def act(self, chat_history: str) -> Tuple[Optional[float], str]:
        """
        Generate the buyer's next move in the negotiation.
        
        Args:
            chat_history: Previous conversation between buyer and seller
            
        Returns:
            Tuple of (price_offer, message)
        """
        # Fill in the buyer prompt with scenario details and history
        prompt = BUYER_PROMPT.format(
            scenario_name=self.scenario['name'],
            item_name=self.scenario['item_name'],
            list_price=self.scenario['list_price'],
            target_price=self.scenario['buyer_target_price'],
            max_price=self.scenario['buyer_max_price'],
            desire_level=self.scenario['buyer_desire_level'],
            chat_history=chat_history
        )
        
        # Get response from AI and parse the price
        response = self._query_llm(prompt)
        return self.parse_price(response)

class ModeratorAgent(Agent):
    """
    Analyzes the finished negotiation and determines who got the better deal.
    
    The moderator has access to all secret information and can provide
    expert analysis of the negotiation tactics and outcome.
    """
    
    def __init__(self, model: str, temp: float, seed: int):
        super().__init__(model, temp, seed)
        print("⚖️ Moderator initialized - Ready to analyze negotiations")

    def analyze(self, transcript: str, final_price: float, scenario: dict) -> str:
        """
        Analyze the completed negotiation and provide expert commentary.
        
        Args:
            transcript: Full conversation between buyer and seller
            final_price: The final agreed price (0 if no deal)
            scenario: The scenario with all secret information
            
        Returns:
            Expert analysis of the negotiation
        """
        # Fill in the moderator prompt with all the details
        prompt = MODERATOR_PROMPT.format(
            item_name=scenario['item_name'],
            final_price=final_price,
            seller_min_price=scenario['seller_min_price'],
            buyer_target_price=scenario['buyer_target_price'],
            buyer_max_price=scenario['buyer_max_price'],
            transcript=transcript
        )
        
        return self._query_llm(prompt)

print("✅ All agent classes created!")
print("\\n🎯 Let's test creating agents for the yard sale scenario:")

# Test creating agents
try:
    test_scenario = get_scenario("yard_sale_lamp")
    
    seller = SellerAgent("phi3", DEFAULT_TEMP, DEFAULT_SEED, test_scenario)
    buyer = BuyerAgent("phi3", DEFAULT_TEMP, DEFAULT_SEED + 1, test_scenario)
    moderator = ModeratorAgent("phi3", DEFAULT_TEMP, DEFAULT_SEED + 2)
    
    print("\\n✅ All agents created successfully!")
    
except Exception as e:
    print(f"❌ Error creating agents: {e}")
    print("Make sure you have Ollama running and the phi3 model installed!")


## Part 5: Running a Simple Negotiation 🎪

Now let's put everything together and run a real negotiation! We'll create a simple function that orchestrates the entire process from start to finish.


In [ ]:
class JsonLogger:
    """
    Simple logger to track the negotiation process.
    
    This helps us save and analyze negotiations later.
    """
    def __init__(self):
        self.log_records = []
    
    def log(self, record: Dict[str, Any]):
        """Add a record to the log with timestamp"""
        timestamped_record = {
            "timestamp": datetime.now().isoformat(),
            **record
        }
        self.log_records.append(timestamped_record)

def format_chat_history(log_records: list) -> str:
    """
    Convert log records into a readable chat history for the agents.
    
    This gives agents context about what has been said so far.
    """
    history_lines = []
    for record in log_records:
        if record.get('type') == 'negotiation_turn':
            role = record['role']
            message = record['message']
            history_lines.append(f"{role}: {message}")
    
    return "\\n".join(history_lines)

def run_simple_negotiation(scenario_key: str, max_rounds: int = 3, model: str = "phi3"):
    """
    Run a complete negotiation between buyer and seller agents.
    
    Args:
        scenario_key: Which scenario to use (e.g., 'yard_sale_lamp')
        max_rounds: Maximum number of negotiation rounds
        model: AI model to use for all agents
    
    Returns:
        Dictionary with negotiation results
    """
    
    # Get the scenario
    scenario = get_scenario(scenario_key)
    if not scenario:
        print(f"❌ Scenario '{scenario_key}' not found!")
        return None
    
    print(f"\\n🎭 Starting Negotiation: '{scenario['item_name']}' at a {scenario['name']}")
    print(f"💰 Listed Price: ${scenario['list_price']}")
    print("=" * 60)
    
    # Initialize logger and agents
    logger = JsonLogger()
    logger.log({"event": "negotiation_start", "scenario": scenario})
    
    # Create agents with different seeds for variety
    seller = SellerAgent(model, DEFAULT_TEMP, DEFAULT_SEED, scenario)
    buyer = BuyerAgent(model, DEFAULT_TEMP, DEFAULT_SEED + 1, scenario)
    moderator = ModeratorAgent(model, DEFAULT_TEMP, DEFAULT_SEED + 2)
    
    # Track negotiation state
    chat_history = ""
    last_buyer_offer = 0
    last_seller_offer = scenario['list_price']
    final_price = 0
    deal_made = False
    
    # Main negotiation loop
    for round_num in range(max_rounds * 2):  # Each round has seller and buyer turn
        turn_type = "Seller" if round_num % 2 == 0 else "Buyer"
        
        if deal_made:
            break
        
        print(f"\\n--- Round {round_num//2 + 1}: {turn_type}'s Turn ---")
        
        # Update chat history for context
        chat_history = format_chat_history(logger.log_records)
        
        if turn_type == "Seller":
            # Seller's turn
            price, message = seller.act(chat_history)
            last_seller_offer = price if price is not None else last_seller_offer
            
            print(f"💰 Seller says: {message}")
            print(f"💰 Seller's Price: ${last_seller_offer}")
            
            logger.log({
                "role": "Seller", 
                "message": message, 
                "price": last_seller_offer, 
                "type": "negotiation_turn"
            })
            
        else:
            # Buyer's turn
            price, message = buyer.act(chat_history)
            last_buyer_offer = price if price is not None else last_buyer_offer
            
            print(f"🛒 Buyer says: {message}")
            print(f"🛒 Buyer's Offer: ${last_buyer_offer}")
            
            logger.log({
                "role": "Buyer", 
                "message": message, 
                "price": last_buyer_offer, 
                "type": "negotiation_turn"
            })
        
        # Check if a deal can be made
        if last_buyer_offer >= last_seller_offer:
            deal_made = True
            final_price = last_seller_offer  # Deal made at seller's asking price
            print(f"\\n🎉 DEAL! Agreement reached at ${final_price}")
            logger.log({"event": "deal_made", "price": final_price})
            break
    
    # Handle case where no deal was reached
    if not deal_made:
        print(f"\\n💔 NO DEAL! Negotiation ended without agreement.")
        print(f"   Final gap: Buyer offered ${last_buyer_offer}, Seller wanted ${last_seller_offer}")
        logger.log({"event": "no_deal"})
    
    # Get moderator analysis
    print("\\n" + "="*60)
    print("⚖️ MODERATOR'S ANALYSIS:")
    print("="*60)
    
    transcript = format_chat_history(logger.log_records)
    analysis = moderator.analyze(transcript, final_price, scenario)
    print(analysis)
    
    logger.log({"role": "Moderator", "message": analysis, "type": "analysis"})
    
    # Return results
    return {
        "scenario": scenario,
        "deal_made": deal_made,
        "final_price": final_price,
        "buyer_offer": last_buyer_offer,
        "seller_offer": last_seller_offer,
        "analysis": analysis,
        "log": logger.log_records
    }

# Let's run a negotiation!
print("🚀 Running a sample negotiation...")
print("Note: This may take a minute as the AI agents think and respond...")

try:
    result = run_simple_negotiation("yard_sale_lamp", max_rounds=2, model="phi3")
    
    if result:
        print("\\n✅ Negotiation completed!")
        print(f"📊 Result: {'Deal at $' + str(result['final_price']) if result['deal_made'] else 'No Deal'}")
    
except Exception as e:
    print(f"❌ Error running negotiation: {e}")
    print("Make sure Ollama is running and you have the model installed!")


## Part 6: Experiment and Analyze! 🔬

Now it's your turn to experiment! Let's create some interactive functions to try different scenarios and analyze the results.


In [ ]:
def analyze_negotiation_outcome(result: dict):
    """
    Provide detailed analysis of a negotiation result.
    
    This helps us understand the dynamics and learn from the negotiation.
    """
    if not result:
        print("❌ No result to analyze!")
        return
    
    scenario = result['scenario']
    
    print("\\n📊 NEGOTIATION ANALYSIS:")
    print("=" * 50)
    
    # Basic outcome
    print(f"🎭 Scenario: {scenario['name']} - {scenario['item_name']}")
    print(f"💰 Listed Price: ${scenario['list_price']}")
    print(f"📈 Deal Made: {'Yes' if result['deal_made'] else 'No'}")
    
    if result['deal_made']:
        print(f"🤝 Final Price: ${result['final_price']}")
        
        # Calculate how well each party did
        seller_min = scenario['seller_min_price']
        buyer_max = scenario['buyer_max_price']
        buyer_target = scenario['buyer_target_price']
        
        seller_profit = result['final_price'] - seller_min
        seller_potential = scenario['list_price'] - seller_min
        seller_success = (seller_profit / seller_potential) * 100 if seller_potential > 0 else 0
        
        buyer_savings = buyer_max - result['final_price']
        buyer_potential = buyer_max - buyer_target
        buyer_success = (buyer_savings / buyer_potential) * 100 if buyer_potential > 0 else 0
        
        print(f"\\n🎯 Performance Analysis:")
        print(f"   💰 Seller Success: {seller_success:.1f}% (${seller_profit} above minimum)")
        print(f"   🛒 Buyer Success: {buyer_success:.1f}% (${buyer_savings} below maximum)")
        
        if seller_success > buyer_success:
            print("   🏆 Seller got the better deal!")
        elif buyer_success > seller_success:
            print("   🏆 Buyer got the better deal!")
        else:
            print("   🤝 Fairly balanced negotiation!")
    
    else:
        print(f"💔 No Deal - Final gap: ${result['seller_offer'] - result['buyer_offer']}")
        print(f"   🛒 Buyer's final offer: ${result['buyer_offer']}")
        print(f"   💰 Seller's final ask: ${result['seller_offer']}")
    
    print("\\n" + "="*50)

def run_multiple_scenarios():
    """
    Run negotiations on all available scenarios for comparison.
    """
    print("🎪 Running negotiations on all scenarios...")
    
    results = {}
    
    for scenario_key in NEGOTIATION_SCENARIOS.keys():
        print(f"\\n{'='*60}")
        print(f"Testing scenario: {scenario_key}")
        print('='*60)
        
        try:
            result = run_simple_negotiation(scenario_key, max_rounds=2, model="phi3")
            results[scenario_key] = result
            
            if result:
                analyze_negotiation_outcome(result)
        
        except Exception as e:
            print(f"❌ Error in {scenario_key}: {e}")
            results[scenario_key] = None
    
    return results

def create_custom_scenario():
    """
    Let users create their own negotiation scenario.
    """
    print("🎨 Create Your Own Negotiation Scenario!")
    print("=" * 50)
    
    # Get user input for custom scenario
    name = input("Enter scenario name (e.g., 'Farmer's Market'): ").strip()
    item = input("Enter item being sold (e.g., 'organic tomatoes'): ").strip()
    
    try:
        list_price = float(input("Enter listed price: $"))
        seller_min = float(input("Enter seller's minimum price (secret): $"))
        buyer_target = float(input("Enter buyer's target price (secret): $"))
        buyer_max = float(input("Enter buyer's maximum price (secret): $"))
    except ValueError:
        print("❌ Invalid price entered!")
        return None
    
    personality = input("Enter seller personality (e.g., 'rushed but fair'): ").strip()
    desire = input("Enter buyer desire level (low/medium/high/very high): ").strip()
    
    custom_scenario = {
        "name": name,
        "item_name": item,
        "list_price": list_price,
        "seller_personality": personality,
        "seller_min_price": seller_min,
        "buyer_desire_level": desire,
        "buyer_target_price": buyer_target,
        "buyer_max_price": buyer_max,
    }
    
    print(f"\\n✅ Custom scenario created!")
    print(json.dumps(custom_scenario, indent=2))
    
    # Add to our scenarios temporarily
    NEGOTIATION_SCENARIOS['custom'] = custom_scenario
    
    return custom_scenario

# Let's try some experiments!
print("🧪 Experiment Options:")
print("1. Analyze the previous negotiation")
print("2. Run all scenarios")  
print("3. Create a custom scenario")
print("\\nChoose what you'd like to try:")

# Analyze the previous result if we have one
if 'result' in locals() and result:
    analyze_negotiation_outcome(result)

# Uncomment the lines below to try different experiments:

# Run all scenarios (this will take a while!)
# all_results = run_multiple_scenarios()

# Create a custom scenario
# custom = create_custom_scenario()
# if custom:
#     custom_result = run_simple_negotiation('custom', max_rounds=3)

print("\\n💡 Try uncommenting the experiment lines above to explore more!")


## Part 7: Advanced Features & Extensions 🚀

Now that you understand the basics, let's explore some advanced concepts and ideas for extending the system!


In [ ]:
# Advanced concepts and extensions you can explore:

print("🚀 ADVANCED NEGOTIATION CONCEPTS:")
print("=" * 60)

print("""
1. 🎭 MULTI-MODEL NEGOTIATIONS
   - Use different AI models for buyer vs seller
   - Compare how different models negotiate
   - Example: phi3 vs llama3 vs gemma
   
2. 🧠 MEMORY & LEARNING
   - Agents remember past negotiations
   - Learn from successful strategies
   - Adapt behavior based on opponent
   
3. 🎯 STRATEGIC BEHAVIORS
   - Anchoring (starting with extreme offers)
   - Concession patterns (how much to give each round)
   - Bluffing and deception detection
   
4. 🌐 COMPLEX SCENARIOS
   - Multiple items being negotiated
   - Time pressure (limited rounds)
   - Third-party mediators
   - Group negotiations (multiple buyers/sellers)
   
5. 📊 ADVANCED ANALYSIS
   - Sentiment analysis of messages
   - Strategy classification
   - Prediction of negotiation outcomes
   - Visualization of negotiation patterns
   
6. 🎮 GAMIFICATION
   - Tournament-style competitions
   - Leaderboards for best negotiators
   - Different difficulty levels
   - Human vs AI negotiations
""")

def demonstrate_multi_model_negotiation():
    """
    Example of using different models for buyer and seller.
    """
    print("\\n🤖 Multi-Model Negotiation Example:")
    print("(Using different AI models for each agent)")
    
    scenario = get_scenario("fish_market")
    
    # Create agents with different models
    # Note: Make sure you have these models installed!
    try:
        seller_aggressive = SellerAgent("phi3", 0.9, DEFAULT_SEED, scenario)  # High temp = more creative
        buyer_conservative = BuyerAgent("phi3", 0.3, DEFAULT_SEED + 1, scenario)  # Low temp = more predictable
        
        print("✅ Created agents with different personalities via temperature!")
        print("   💰 Seller: High creativity (temp=0.9)")
        print("   🛒 Buyer: Conservative approach (temp=0.3)")
        
    except Exception as e:
        print(f"❌ Error: {e}")
        print("Make sure you have the required models installed!")

def save_negotiation_results(results: dict, filename: str):
    """
    Save negotiation results to a file for later analysis.
    """
    try:
        with open(filename, 'w') as f:
            json.dump(results, f, indent=2, default=str)
        print(f"✅ Results saved to {filename}")
    except Exception as e:
        print(f"❌ Error saving results: {e}")

# Demonstrate advanced concepts
demonstrate_multi_model_negotiation()

print("\\n" + "="*60)
print("💡 IDEAS FOR YOUR OWN EXTENSIONS:")
print("="*60)

extension_ideas = [
    "🎨 Create a web interface using Streamlit or Flask",
    "📱 Build a mobile app for human vs AI negotiations", 
    "🎵 Add audio generation for spoken negotiations",
    "📈 Create data visualizations of negotiation patterns",
    "🧪 Experiment with different prompt engineering techniques",
    "🎯 Add personality tests to classify negotiation styles",
    "🌍 Create scenarios from different cultures/contexts",
    "🤝 Build multi-party negotiations (3+ participants)",
    "📚 Train agents on real negotiation transcripts",
    "🎮 Create a negotiation training game for students"
]

for i, idea in enumerate(extension_ideas, 1):
    print(f"{i:2d}. {idea}")

print("\\n🎓 LEARNING OBJECTIVES ACHIEVED:")
print("✅ Understanding multi-agent systems")
print("✅ Prompt engineering for different personalities") 
print("✅ Game theory and strategic interactions")
print("✅ System architecture and orchestration")
print("✅ Analysis and evaluation of AI behaviors")
print("✅ Real-world application of AI concepts")

print("\\n🚀 You're now ready to build your own AI negotiation systems!")
